In [0]:
#dbutils.fs.rm('s3://one-env-uc-external-location/aht-sa/ext-data',True)
#display(dbutils.fs.ls('s3://one-env-uc-external-location/aht-sa/ext-data'))

In [0]:
from pyspark.sql.functions import explode, struct, from_json,col,first,arrays_zip,col,map_from_entries,expr
catalog = 'ahtsa'
schema = 'fitbit_workshop'
vol = '/Volumes/ahtsa/fitbit/raw_fitbitapi'
data_path = 's3://one-env-uc-external-location/aht-sa/ext-data'

In [0]:
%sql
use catalog ahtsa; 
use schema fitbit_workshop;

In [0]:
%sql
Select  activities_exploded.*
FROM (SELECT EXPLODE(activities) AS activities_exploded 
FROM JSON.`/Volumes/ahtsa/fitbit/raw_fitbitapi/activitylog`
)

In [0]:
%sql

CREATE TABLE IF NOT EXISTS fitbit_workshop.bronze_activities
LOCATION 's3://one-env-uc-external-location/aht-sa/ext-data/bronze_activities';

COPY INTO fitbit_workshop.bronze_activities 
FROM (
  SELECT EXPLODE(activities) AS activities_exploded FROM '/Volumes/ahtsa/fitbit/raw_fitbitapi/activitylog'
)
FILEFORMAT = JSON FORMAT_OPTIONS ('mergeSchema' = 'true') COPY_OPTIONS ('mergeSchema' = 'true')

In [0]:
#silver load 
silver_df = (
    spark.table('fitbit_workshop.bronze_activities').select('activities_exploded.*')
    .distinct()
    .withColumn(
        "activityLevelMap",
        map_from_entries(
            expr("transform(activityLevel, x -> struct(x.name, x.minutes))")
        )
    )
    .withColumn(
        "heartRateZonesMap",
        map_from_entries(
            expr("transform(heartRateZones, x -> struct(x.name, struct(x.caloriesOut, x.max, x.min, x.minutes)))")
        )
    ).selectExpr(
            "logId", "startTime","activityName","steps","speed","pace",
            "logType","averageHeartRate","calories", "distance", "distanceUnit",
            "duration","hasActiveZoneMinutes","hasGps",
             "activityLevelMap as activityLevels", 
             "heartRateZonesMap as heartRateZones")
)

silver_df.write \
    .mode("overwrite") \
    .option("path", f"{data_path}/silver_activities") \
    .saveAsTable("fitbit_workshop.silver_activities")

In [0]:
%sql
CREATE OR REPLACE FUNCTION fitbit_workshop.calculate_pace(ms DOUBLE, km DOUBLE)
    RETURNS DOUBLE
    NOT DETERMINISTIC
    CONTAINS SQL
    COMMENT 'Returns a minute per mile double value'
    RETURN try_divide(ms/60000,km*0.621371);

In [0]:
%sql
CREATE OR REPLACE TABLE fitbit_workshop.gold_running as (select DISTINCT
date(startTime) as date,
logId, 
activityName, 
calories, 
averageHeartRate, 
distance*0.621371 as miles, 
duration/60000 as min,
calculate_pace(duration,distance) as min_per_mile
from fitbit_workshop.silver_activities
where activityName IN ('Run','Treadmill') AND calculate_pace(duration,distance) <=15
);

select * from fitbit_workshop.gold_running

Databricks visualization. Run in Databricks to view.

In [0]:
import folium
from folium.plugins import PolyLineTextPath

df = spark.sql("select * from fitbit.bronze_tracking where id = '2024-10-20T09:00:57.000-04:00'")

# Convert the Spark DataFrame to a Pandas DataFrame
route_df = df.select("LatitudeDegrees", "LongitudeDegrees").toPandas()

# Create a map centered around the first point
m = folium.Map(location=[route_df["LatitudeDegrees"].iloc[0], route_df["LongitudeDegrees"].iloc[0]], zoom_start=15)

# Create a list of coordinate pairs
route = list(zip(route_df["LatitudeDegrees"], route_df["LongitudeDegrees"]))

# Add the route to the map
folium.PolyLine(route, color="blue", weight=2.5, opacity=1).add_to(m)

# Display the map
display(m)

In [0]:
%sql
select * from ahtsa.fitbit.bronze_tracking